In [1]:
import pandas as pd
import numpy as np

import copy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
def createdf(filename): #створення фрейму
    df = pd.read_csv(filename)
    
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.isnull().any()
    return(df)

In [3]:
filename = "csv//int20h-ds-test-dataset.csv"

In [4]:
df = createdf(filename)

In [5]:
df["event_created_date"] = pd.to_datetime(df["event_created_date"])

In [6]:
print(str(df.shape[0]) +" rows x " + str(df.shape[1]) + " columns")
df.head()

23357 rows x 8 columns


,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
0,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Success,"{""Make"":""Dodge"",""Model"":""Caravan"",""Color"":""Whi...",2022-01-16 17:03:04,android,samsung,SM-N975U
1,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Break,{},2022-01-16 17:07:47,android,samsung,SM-N975U
2,f344be2d9a042b7444f3cc5279e38ef1,FL,Calculator View,{},2022-01-16 17:16:25,android,samsung,SM-G973U1
3,c95c777785faec8dd910d019d7278ebe,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":""""}",2022-01-16 17:24:22,android,samsung,SM-N975U
4,e331ed81422d8fba55520a43a872e701,IL,Sign Up Success,"{""Method"":""Apple""}",2022-01-16 17:34:51,ios,Apple,"iPhone12,1"


In [7]:
#drop users that have not added, renewed or canceled Subscription Premium - they haven't canceled it because they had no sub

def remove_excess_data(df):
    target_values = ['Subscription Premium Cancel', 'Subscription Premium', 'Subscription Premium Renew']
    user_list = df["userid"].unique()
    for i in user_list:
        df_temp = df[df["userid"]==i]
        if target_values[0] in df_temp.values or target_values[1] in df_temp.values or target_values[2] in df_temp.values:
            continue
        df = df.drop(df_temp.index)
    return df

In [8]:
df = remove_excess_data(df)

In [9]:
df = df.sort_values(["userid", "event_created_date"], ascending=[True, True]).reset_index(drop = True)

In [10]:
#Замінюємо невідомі значення пристроїв з яких виконувалися дії 
#користувачів для яких відомий пристрій за іншими діями на ці пристрої
def fill_in_NaNs(df):
    for i in range(len(df)):
        if df.loc[i, "event_platform"] == "outofband":
            df_temp = df[df["userid"] == df.iloc[i]["userid"]]
            for index, row in df_temp.iterrows():
                if row["event_platform"] != "outofband":
                    df.loc[i, "event_platform"] = row.at["event_platform"]
                    df.loc[i, "device_manufacture"] = row.at["device_manufacture"]
                    df.loc[i, "device_model"] = row.at["device_model"]
                    break
    return df


In [11]:
df = fill_in_NaNs(df)
df_canceled = df[df["event_name"] == 'Subscription Premium Cancel']
#print(str(df_canceled.shape[0]) +" rows x " + str(df_canceled.shape[1]) + " columns")
canceled_list = df_canceled["userid"].unique()
df_canceled.head()

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
74,03e0c91e1163e8b80e74e586a3e666d6,MD,Subscription Premium Cancel,{},2022-02-24 19:45:02,ios,Apple,"iPhone12,1"
118,051a5f8bce75a601be987980a09d4187,VA,Subscription Premium Cancel,{},2022-04-06 10:12:37,ios,Apple,"iPhone14,2"
127,05630c0e889fe7371fc74d7ae01f491f,FL,Subscription Premium Cancel,{},2022-03-19 18:16:37,ios,Apple,"iPhone13,2"
137,060153cf7a5844d9aed735d408b35f97,TX,Subscription Premium Cancel,{},2022-02-11 15:06:11,ios,Apple,"iPhone14,2"
157,078ae59f1ae79df514e846e5e51eca8b,CA,Subscription Premium Cancel,{},2022-04-09 12:43:36,ios,Apple,"iPhone12,1"


In [12]:
def event_occured_per_user(row):
    if row['userid'] in event_list:
        return 1
    else:
        return 0

In [13]:
def event_occured(row):
    if row["event_name"] == event:
        return 1
    else:
        return 0

In [14]:
event = 'Subscription Premium Cancel'
df_temp = df[df["event_name"] == event]
event_list = df_temp["userid"].unique()
df[event] = df.apply(event_occured_per_user, axis=1)

In [15]:
df.to_csv("csv//sorted_filled_test.csv")

In [16]:
df = createdf("csv//sorted_filled_test.csv")

In [17]:
#виводимо з значення дати значення місяця, квартал, день тижня та час події в хвилинах
def event_created_date_transform(df):
    date = pd.DataFrame({"time" : []})
    date["time"] = pd.to_datetime(df["event_created_date"])
    date["month"] = date["time"].dt.month
    date["quarter"] = date["time"].dt.quarter
    date["day_week"] = date["time"].dt.dayofweek
    #date["time"] = (date["time"].dt.minute.to_numpy(dtype=np.int32) + date["time"].dt.hour.to_numpy(dtype=np.int32)*60)/(60*24) #minute of the day
    date["time"] = date["time"].dt.hour.to_numpy(dtype=np.int32) #hour of the day
    return date

In [18]:
date = event_created_date_transform(df)

In [19]:
date.to_csv("csv//extracted_date_data.csv")

In [20]:
canceled_list = df_canceled["userid"].unique()

In [21]:
df_time = copy.deepcopy(df)

In [22]:
event = 'Subscription Premium Cancel'
df_time[event] = df_time.apply(event_occured, axis=1)

In [23]:
df_time = df_time.drop(['userid', 'event_name', 'event_attributes', 'event_created_date', "user_state", "event_platform","device_manufacture", "device_model"], axis=1)

In [24]:
df_time = pd.concat([df_time, date], axis=1)

In [25]:
df_time.to_csv("csv//event_and_date_data.csv")

In [26]:
df_event_occured = copy.deepcopy(df)
events = df["event_name"].unique()

In [27]:
for e in events:
    event = e
    df_temp = df_event_occured[df_event_occured["event_name"] == event]
    event_list = df_temp["userid"].unique()
    df_event_occured[event] = df_event_occured.apply(event_occured_per_user, axis=1)

In [28]:
df_event_occured = df_event_occured.drop(['event_name', 'event_attributes', 'event_created_date', "user_state", "event_platform","device_manufacture", "device_model"], axis=1)

In [29]:
df_event_occured = df_event_occured.drop_duplicates(subset = ["userid"])

In [30]:

#fig, ax = plt.subplots(figsize=(15,15))    
#dataplot = sns.heatmap(df_event_occured.drop(["userid"], axis = 1).corr(), cmap="RdYlBu", annot=True, ax = ax)
#plt.show()

In [31]:
df_event_occured.to_csv("csv//event_occured_per_user.csv")

In [32]:
df2 = copy.deepcopy(df)

In [33]:
label_encoder = LabelEncoder()
df_static_data = copy.deepcopy(df2.drop(['userid', 'event_name', 'event_attributes', 'event_created_date'], axis=1))


In [34]:
categories = ["user_state", "event_platform","device_manufacture", "device_model"]
for category in categories:
    #print(df_static_data[category].unique())
    df_static_data[category]=label_encoder.fit_transform(df_static_data[category])
    #print(df_static_data[category].unique())


In [35]:
#dataplot = sns.heatmap(df_static_data.corr(), cmap="RdYlBu", annot=True)
#plt.show()

In [36]:
df_static_data.to_csv("csv//static_data.csv")